In [14]:
!pip install requests
!pip install bs4
!pip install lxml
!pip install pandas

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.2 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 542.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ---------- ---------------------------- 92.2/341.8 kB 2.6 MB/s eta 0:00:01
     ------------------- ------------------ 174.1/341.8 kB 2.1 MB/s eta 0:00:01
     -------------------------------------  337.9/341.8 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 341.8/341.8 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.4/10.6 MB 13.5 MB/s eta 0:00:01
   --- ------------------------------------ 0.8/10.6 MB 8.9 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/10.6 MB 8.5 MB/s eta 0:00:02


In [15]:
# import libraries
import os
import pandas as pd
import lxml
import requests  
from bs4 import BeautifulSoup

In [ ]:
def read_txt_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            return lines
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

match_url_list = read_txt_file(os.getcwd() + '\data\Ascension.txt')

In [ ]:
# In order to get the list of match of the all VLR Season
match_url_list = []

source_matchlist = requests.get(url='https://www.vlr.gg/event/matches/449/valorant-champions/?series_id=all').text

soup_matchlist = BeautifulSoup(source_matchlist,'lxml')

days = soup_matchlist.findAll('div', {'class':'wf-card'})

for d in range(1,len(days)):
    for i in days[d].findAll(href=True):
        match_url_list.append(i['href'])

In [62]:
match_stats = []

for matchnum in range(len(match_url_list)):
    url = match_url_list[matchnum]
    
    source_match = requests.get(url=url).text
    soup_match = BeautifulSoup(source_match, features="html.parser")
    
    table_match = soup_match.findAll('table', {'class':'wf-table-inset mod-overview'})

    stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]

    series = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split("\n", 1)[1].strip()
    
    for a in [a for a in list(range(len(table_match))) if a not in [2,3]]:
        if a in [0,1]:
            map_num = 1
        else:
            map_num = a//2
        
        map_name = soup_match.findAll('div', {'class':'vm-stats-gamesnav-item js-map-switch'})[map_num-1].text.strip()[1:].strip()
        stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]
        team_name = soup_match.findAll('div', {'class':'wf-title-med'})[a % 2].text.strip()
        
        table = table_match[a]
        headers_match = []
        
        for i in table.find_all('th'):
            title_match = i.text.strip()
            headers_match.append(title_match)
            
        headers_match[0] = 'Player Name'
        headers_match[1] = 'Agent Name'
        headers_match[6] = 'K/D +/–'   
        headers_match[12] = 'FK/FD +/–'   
        df_match = pd.DataFrame(columns=headers_match)
        
        for row in table.find_all('tr')[1:]:
            data = row.find_all('td')
            row_data = [td.text.strip() for td in data]
            row_data[1] = str(data[1]).split('title="',1)[1].split('"',1)[0].title()
            length = len(df_match)
            df_match.loc[length] = row_data
        
        df_match['Team Name'] = team_name
        df_match['Map Name'] = map_name
        df_match['Map #'] = map_num
        df_match['Stage'] = stage
        df_match['Series'] = series
        
        match_stats.append(df_match)
    

result = pd.concat(match_stats).reset_index(drop=True)
result['Player Name'] = result['Player Name'].str.split("\n").str[0].str.strip()
result['D'] = result['D'].str[1:-1]
result = result.apply(pd.to_numeric, errors='ignore')
result.drop('KAST', axis=1, inplace = True)

result.to_csv('data/stat_match_ascension.csv')

In [59]:
Grand_final
Magnum = Grand_final[Grand_final['Player Name'] == 'MAGNUM']

In [60]:
Magnum

,Player Name,Agent Name,R,ACS,K,D,K/D +/–,+/–,ADR,HS%,FK,FK/FD +/–,+/–,Team Name,Map Name,Map #,Stage,Series
0,MAGNUM,Killjoy,0.98\n0.98\n0.98,225\n237\n206,15\n10\n5,\n\n14\n9\n5\n\n,1\n1\n0,+1\n+1\n0,153\n164\n134,24%\n28%\n19%,5\n4\n1,3\n1\n2,+2\n+3\n-1,Apeks,Haven,1,Playoffs,Grand Final
14,MAGNUM,Cypher,0.67\n0.41\n0.92,120\n62\n174,8\n1\n7,\n\n17\n9\n8\n\n,6\n3\n3,-9\n-8\n-1,85\n44\n121,13%\n0%\n21%,3\n1\n2,2\n2\n0,+1\n-1\n+2,Apeks,Ascent,2,Playoffs,Grand Final
21,MAGNUM,Cypher,1.03\n1.28\n0.76,227\n259\n193,18\n11\n7,\n\n18\n8\n10\n\n,5\n3\n2,0\n+3\n-3,147\n157\n137,22%\n28%\n16%,6\n3\n3,4\n1\n3,+2\n+2\n0,Apeks,Pearl,3,Playoffs,Grand Final


In [61]:
Grand_final

,Player Name,Agent Name,R,ACS,K,D,K/D +/–,+/–,ADR,HS%,FK,FK/FD +/–,+/–,Team Name,Map Name,Map #,Stage,Series
0,MAGNUM,Killjoy,0.98\n0.98\n0.98,225\n237\n206,15\n10\n5,\n\n14\n9\n5\n\n,1\n1\n0,+1\n+1\n0,153\n164\n134,24%\n28%\n19%,5\n4\n1,3\n1\n2,+2\n+3\n-1,Apeks,Haven,1,Playoffs,Grand Final
1,Enzo,Sova,0.94\n0.75\n1.26,191\n132\n293,10\n4\n6,\n\n14\n8\n6\n\n,10\n5\n5,-4\n-4\n0,123\n100\n163,18%\n21%\n15%,2\n0\n2,2\n2\n0,0\n-2\n+2,Apeks,Haven,1,Playoffs,Grand Final
2,ShadoW,Breach,0.85\n0.93\n0.73,165\n174\n152,12\n9\n3,\n\n14\n8\n6\n\n,11\n9\n2,-2\n+1\n-3,89\n92\n84,23%\n35%\n8%,1\n1\n0,1\n1\n0,0\n0\n0,Apeks,Haven,1,Playoffs,Grand Final
3,Keiko,Jett,0.84\n1.13\n0.35,214\n278\n107,14\n12\n2,\n\n16\n9\n7\n\n,3\n2\n1,-2\n+3\n-5,151\n191\n81,26%\n27%\n22%,2\n1\n1,0\n0\n0,+2\n+1\n+1,Apeks,Haven,1,Playoffs,Grand Final
4,Mistic,Omen,0.70\n0.53\n0.98,130\n117\n155,8\n4\n4,\n\n15\n10\n5\n\n,4\n1\n3,-7\n-6\n-1,93\n79\n117,24%\n19%\n33%,1\n1\n0,2\n1\n1,-1\n0\n-1,Apeks,Haven,1,Playoffs,Grand Final
5,logaN,Killjoy,1.50\n1.50\n1.50,265\n302\n244,20\n9\n11,\n\n9\n3\n6\n\n,3\n2\n1,+11\n+6\n+5,168\n186\n157,20%\n29%\n15%,1\n0\n1,1\n0\n1,0\n0\n0,Gentle Mates,Haven,1,Playoffs,Grand Final
6,Wailers,Omen,1.20\n1.27\n1.16,194\n228\n176,13\n5\n8,\n\n11\n5\n6\n\n,12\n6\n6,+2\n0\n+2,133\n154\n121,17%\n15%\n19%,2\n1\n1,3\n0\n3,-1\n+1\n-2,Gentle Mates,Haven,1,Playoffs,Grand Final
7,nataNk,Breach,1.15\n0.88\n1.31,220\n206\n229,15\n5\n10,\n\n13\n5\n8\n\n,9\n4\n5,+2\n0\n+2,142\n125\n151,25%\n29%\n24%,2\n0\n2,5\n2\n3,-3\n-2\n-1,Gentle Mates,Haven,1,Playoffs,Grand Final
8,TakaS,Jett,1.09\n1.58\n0.79,205\n273\n166,15\n8\n7,\n\n14\n3\n11\n\n,3\n1\n2,+1\n+5\n-4,136\n166\n119,22%\n24%\n20%,2\n2\n0,1\n1\n0,+1\n+1\n0,Gentle Mates,Haven,1,Playoffs,Grand Final
9,beyAz,Sova,0.85\n0.64\n0.97,164\n102\n201,10\n2\n8,\n\n12\n5\n7\n\n,6\n3\n3,-2\n-3\n+1,108\n75\n127,19%\n29%\n16%,1\n0\n1,1\n1\n0,0\n-1\n+1,Gentle Mates,Haven,1,Playoffs,Grand Final
